In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from datetime import datetime
import numpy as np

**Соберите данные из Google Sheets и напишите функцию `generate_report`, которая принимает три списка (данные с каждого листа в Google Sheets) и сохраняет отчет в файл `student_debt_report.txt`**

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [ ]:
!wget https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json

!wget https://gist.github.com/Vs8th/39c5deed0f5539d781f00328f7fd4fe0/raw/result.txt

--2024-06-24 05:41:14--  https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.github.com (gist.github.com)... 140.82.113.4
Connecting to gist.github.com (gist.github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json [following]
--2024-06-24 05:41:15--  https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2358 (2.3K) [text/plain]
Saving to: ‘creds.json’

creds.json          100%[===================>]   2.30K  --.-KB/s    in 0s      

2024-06-24 05:41:15 (15.4 MB/s) - ‘creds.json’ saved [2

Чтобы посмотреть как выглядят сами гугл таблицы в оригинале - можете перейти по [ссылке](https://docs.google.com/spreadsheets/d/1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U/edit?usp=sharing) и изучить.

Как примерно должен выглядеть результат:

**Примечание**

Считать должников необходимо на 1 марта 2023 года. То есть определяя количество просроченных платежей, мы определяем их количество не на настоящую дату, а на **1 марта 2023 года**. А периоды внесения платежей для всех студентов одинаковы - **6 месяцев**.

То есть, если ожидаемый платеж должен был пройти 3 февраля 2022 года, то к 1 марта 2023 студент просрочил уже три платежа (3 февраля 2022, 3 августа 2022 и 3 февраля 2023).

**Решение**

Напишите свое решение ниже

In [ ]:
# Указываем необходимые права доступа к таблицам
scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']

# Загружаем ключи аутентификации из файла json
creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)

# Авторизуемся в Google Sheets API
client = gspread.authorize(creds)

In [ ]:
sheet1 = client.open("Installments").worksheet("Лист1")
sheet1_data = sheet1.get_all_records()
df1 = pd.DataFrame(sheet1_data)
df1

,student_id,student_name,installment
0,1,Смирнова И.И.,Y
1,2,Кузнецова К.А.,Y
2,3,Петров К.А.,Y
3,4,Смирнова А.А.,Y
4,5,Сидоров К.А.,N
...,...,...,...
495,496,Кузнецова И.И.,N
496,497,Иванов И.И.,N
497,498,Николаева А.А.,N
498,499,Сидоров П.П.,Y


In [ ]:
sheet2 = client.open("Installments").worksheet("Лист2")
sheet2_data = sheet2.get_all_records()
df2 = pd.DataFrame(sheet2_data)
df2

,student_id,last_payment_date,expected_payment_date
0,1,20.09.2022,19.03.2023
1,2,09.08.2022,05.02.2023
2,3,15.02.2022,14.08.2022
3,4,22.12.2022,20.06.2023
4,6,29.07.2022,25.01.2023
...,...,...,...
248,490,02.06.2022,02.12.2022
249,493,30.09.2022,29.03.2023
250,494,12.02.2022,12.08.2022
251,495,29.04.2022,29.10.2022


In [ ]:
sheet3 = client.open("Installments").worksheet("Лист3")
sheet3_data = sheet3.get_all_records()
df3 = pd.DataFrame(sheet3_data)
df3

,student_id,already_payed_amount,left_to_pay,one-time_payment,installment_amount
0,1,122373,277627,133333,400000
1,2,325610,474390,266666,800000
2,3,51843,148157,33333,200000
3,4,19656,480344,41666,500000
4,6,18723,581277,100000,600000
...,...,...,...,...,...
248,490,45836,254164,25000,300000
249,493,14272,185728,16666,200000
250,494,312773,387227,233333,700000
251,495,25117,174883,33333,200000


In [ ]:
df_merge = df1.merge(df2, how='left', on='student_id').merge(df3, how='left', on='student_id')
df_merge

,student_id,student_name,installment,last_payment_date,expected_payment_date,already_payed_amount,left_to_pay,one-time_payment,installment_amount
0,1,Смирнова И.И.,Y,20.09.2022,19.03.2023,122373.0,277627.0,133333.0,400000.0
1,2,Кузнецова К.А.,Y,09.08.2022,05.02.2023,325610.0,474390.0,266666.0,800000.0
2,3,Петров К.А.,Y,15.02.2022,14.08.2022,51843.0,148157.0,33333.0,200000.0
3,4,Смирнова А.А.,Y,22.12.2022,20.06.2023,19656.0,480344.0,41666.0,500000.0
4,5,Сидоров К.А.,N,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
495,496,Кузнецова И.И.,N,NaN,NaN,NaN,NaN,NaN,NaN
496,497,Иванов И.И.,N,NaN,NaN,NaN,NaN,NaN,NaN
497,498,Николаева А.А.,N,NaN,NaN,NaN,NaN,NaN,NaN
498,499,Сидоров П.П.,Y,12.04.2022,12.10.2022,232915.0,767085.0,333333.0,1000000.0


In [ ]:
df_instm = df_merge[df_merge['installment'] == 'Y']
df_instm.isna().any()

student_id               False
student_name             False
installment              False
last_payment_date        False
expected_payment_date    False
already_payed_amount     False
left_to_pay              False
one-time_payment         False
installment_amount       False
dtype: bool

In [ ]:
df_instm

,student_id,student_name,installment,last_payment_date,expected_payment_date,already_payed_amount,left_to_pay,one-time_payment,installment_amount
0,1,Смирнова И.И.,Y,20.09.2022,19.03.2023,122373.0,277627.0,133333.0,400000.0
1,2,Кузнецова К.А.,Y,09.08.2022,05.02.2023,325610.0,474390.0,266666.0,800000.0
2,3,Петров К.А.,Y,15.02.2022,14.08.2022,51843.0,148157.0,33333.0,200000.0
3,4,Смирнова А.А.,Y,22.12.2022,20.06.2023,19656.0,480344.0,41666.0,500000.0
5,6,Иванов А.А.,Y,29.07.2022,25.01.2023,18723.0,581277.0,100000.0,600000.0
...,...,...,...,...,...,...,...,...,...
489,490,Иванов М.А.,Y,02.06.2022,02.12.2022,45836.0,254164.0,25000.0,300000.0
492,493,Иванов И.И.,Y,30.09.2022,29.03.2023,14272.0,185728.0,16666.0,200000.0
493,494,Иванов М.А.,Y,12.02.2022,12.08.2022,312773.0,387227.0,233333.0,700000.0
494,495,Николаева М.А.,Y,29.04.2022,29.10.2022,25117.0,174883.0,33333.0,200000.0


In [ ]:
df_instm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 253 entries, 0 to 498
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   student_id             253 non-null    int64  
 1   student_name           253 non-null    object 
 2   installment            253 non-null    object 
 3   last_payment_date      253 non-null    object 
 4   expected_payment_date  253 non-null    object 
 5   already_payed_amount   253 non-null    float64
 6   left_to_pay            253 non-null    float64
 7   one-time_payment       253 non-null    float64
 8   installment_amount     253 non-null    float64
dtypes: float64(4), int64(1), object(4)
memory usage: 27.9+ KB


In [ ]:
df_instm['last_payment_date'] = pd.to_datetime(df_instm['last_payment_date'])
df_instm['expected_payment_date'] = pd.to_datetime(df_instm['expected_payment_date'])

<ipython-input-16-a4cae09fc4e5>:1: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_instm['last_payment_date'] = pd.to_datetime(df_instm['last_payment_date'])
<ipython-input-16-a4cae09fc4e5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_instm['last_payment_date'] = pd.to_datetime(df_instm['last_payment_date'])
<ipython-input-16-a4cae09fc4e5>:2: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_instm['expected_payment_date'] = pd.to_datetime(df_instm['expected_payment_date'])
<ipython-input-16-a4cae09fc4e5>:2: Se

In [ ]:
df_instm = df_instm[df_instm['expected_payment_date'] < datetime(2023, 3, 1)]
df_instm

,student_id,student_name,installment,last_payment_date,expected_payment_date,already_payed_amount,left_to_pay,one-time_payment,installment_amount,current_debt
1,2,Кузнецова К.А.,Y,2022-08-09,2023-02-05,325610.0,474390.0,266666.0,800000.0,266666
2,3,Петров К.А.,Y,2022-02-15,2022-08-14,51843.0,148157.0,33333.0,200000.0,66666
5,6,Иванов А.А.,Y,2022-07-29,2023-01-25,18723.0,581277.0,100000.0,600000.0,100000
9,10,Иванов П.П.,Y,2022-06-23,2022-12-20,52263.0,647737.0,116666.0,700000.0,116666
11,12,Петров И.И.,Y,2022-08-11,2023-02-07,174708.0,525292.0,116666.0,700000.0,116666
...,...,...,...,...,...,...,...,...,...,...
482,483,Сидоров А.А.,Y,2022-06-12,2022-12-12,520614.0,279386.0,266666.0,800000.0,266666
489,490,Иванов М.А.,Y,2022-06-02,2022-12-02,45836.0,254164.0,25000.0,300000.0,25000
493,494,Иванов М.А.,Y,2022-02-12,2022-08-12,312773.0,387227.0,233333.0,700000.0,387227
494,495,Николаева М.А.,Y,2022-04-29,2022-10-29,25117.0,174883.0,33333.0,200000.0,33333


In [ ]:
df_instm['current_debt'] = df_instm.apply(lambda row: row['one-time_payment']  if row['expected_payment_date'] >= datetime(2022, 8, 31)\
                                          else row['one-time_payment'] * 2, axis=1)
df_instm['current_debt'] = df_instm.apply(lambda row: row['current_debt']  if row['current_debt'] < row['left_to_pay']\
                                          else row['left_to_pay'], axis=1)\
                                          .astype(int)
df_instm

<ipython-input-33-09bf52de7552>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_instm['current_debt'] = df_instm.apply(lambda row: row['one-time_payment']  if row['expected_payment_date'] >= datetime(2022, 8, 31)\
<ipython-input-33-09bf52de7552>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_instm['current_debt'] = df_instm.apply(lambda row: row['current_debt']  if row['current_debt'] < row['left_to_pay']\


,student_id,student_name,installment,last_payment_date,expected_payment_date,already_payed_amount,left_to_pay,one-time_payment,installment_amount,current_debt
1,2,Кузнецова К.А.,Y,2022-08-09,2023-02-05,325610.0,474390.0,266666.0,800000.0,266666
2,3,Петров К.А.,Y,2022-02-15,2022-08-14,51843.0,148157.0,33333.0,200000.0,66666
5,6,Иванов А.А.,Y,2022-07-29,2023-01-25,18723.0,581277.0,100000.0,600000.0,100000
9,10,Иванов П.П.,Y,2022-06-23,2022-12-20,52263.0,647737.0,116666.0,700000.0,116666
11,12,Петров И.И.,Y,2022-08-11,2023-02-07,174708.0,525292.0,116666.0,700000.0,116666
...,...,...,...,...,...,...,...,...,...,...
482,483,Сидоров А.А.,Y,2022-06-12,2022-12-12,520614.0,279386.0,266666.0,800000.0,266666
489,490,Иванов М.А.,Y,2022-06-02,2022-12-02,45836.0,254164.0,25000.0,300000.0,25000
493,494,Иванов М.А.,Y,2022-02-12,2022-08-12,312773.0,387227.0,233333.0,700000.0,387227
494,495,Николаева М.А.,Y,2022-04-29,2022-10-29,25117.0,174883.0,33333.0,200000.0,33333


In [ ]:
for i, row in df_instm.iterrows():
  print(f"Студент {row['student_name']} - долг {row['current_debt']} рублей")

Студент Кузнецова К.А. - долг 266666 рублей
Студент Петров К.А. - долг 66666 рублей
Студент Иванов А.А. - долг 100000 рублей
Студент Иванов П.П. - долг 116666 рублей
Студент Петров И.И. - долг 116666 рублей
Студент Петров А.А. - долг 16666 рублей
Студент Иванов Е.В. - долг 41666 рублей
Студент Николаева Е.В. - долг 100000 рублей
Студент Смирнова Е.В. - долг 233333 рублей
Студент Николаева М.А. - долг 33332 рублей
Студент Кузнецова А.А. - долг 133332 рублей
Студент Иванов А.А. - долг 25000 рублей
Студент Петров И.И. - долг 133332 рублей
Студент Кузнецова К.А. - долг 133333 рублей
Студент Кузнецова Е.В. - долг 66666 рублей
Студент Сидоров И.И. - долг 83333 рублей
Студент Петров И.И. - долг 16666 рублей
Студент Кузнецова И.И. - долг 233333 рублей
Студент Иванов А.А. - долг 266666 рублей
Студент Сидоров К.А. - долг 266666 рублей
Студент Смирнова И.И. - долг 200000 рублей
Студент Сидоров М.А. - долг 166666 рублей
Студент Сидоров Е.В. - долг 33333 рублей
Студент Иванов Е.В. - долг 100000 руб

In [ ]:
with open('student_debt_report.txt', 'w') as file:
  for i, row in df_instm.iterrows():
    file.write(f"Студент {row['student_name']} - долг {row['current_debt']} рублей\n")

In [ ]:
def generate_report(sheet1, sheet2, sheet3):
  df1 = pd.DataFrame(sheet1_data)
  df2 = pd.DataFrame(sheet2_data)
  df3 = pd.DataFrame(sheet3_data)
  df_merge = df1.merge(df2, how='left', on='student_id').merge(df3, how='left', on='student_id')

  df_instm = df_merge[df_merge['installment'] == 'Y']

  df_instm['last_payment_date'] = pd.to_datetime(df_instm['last_payment_date'])
  df_instm['expected_payment_date'] = pd.to_datetime(df_instm['expected_payment_date'])

  df_instm = df_instm[df_instm['expected_payment_date'] <= datetime(2023, 2, 28)]

  df_instm['current_debt'] = df_instm.apply(lambda row: row['one-time_payment']  if row['expected_payment_date']\
                                            >= datetime(2022, 8, 31)\
                                            else row['one-time_payment'] * 2, axis=1)
  df_instm['current_debt'] = df_instm.apply(lambda row: row['current_debt']  if row['current_debt'] < row['left_to_pay']\
                                          else row['left_to_pay'], axis=1)\
                                          .astype(int)

  with open('student_debt_report.txt', 'w') as file:
    for i, row in df_instm.iterrows():
      file.write(f"Студент {row['student_name']} - долг {row['current_debt']} рублей\n")

In [ ]:
generate_report(sheet1, sheet2, sheet3)

<ipython-input-53-c1336b2b13dc>:9: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_instm['last_payment_date'] = pd.to_datetime(df_instm['last_payment_date'])
<ipython-input-53-c1336b2b13dc>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_instm['last_payment_date'] = pd.to_datetime(df_instm['last_payment_date'])
<ipython-input-53-c1336b2b13dc>:10: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_instm['expected_payment_date'] = pd.to_datetime(df_instm['expected_payment_date'])
<ipython-input-53-c1336b2b13dc>:10: 

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt

import pandas as pd

user_answer = pd.read_csv('student_debt_report.txt')
correct_answer = pd.read_csv('student_debt.txt')

--2024-06-24 06:05:25--  https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.github.com (gist.github.com)... 140.82.114.3
Connecting to gist.github.com (gist.github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt [following]
--2024-06-24 06:05:25--  https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11325 (11K) [text/plain]
Saving to: ‘student_debt.txt’

student_debt.txt    100%[===================>]  11.06K  --.-KB/s    in 0s      

2024-06-24 06:05:25 (74.3 MB/s)

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')